In [26]:
from pathlib import Path

project_dir = Path('.').absolute().parent
project_dir

PosixPath('/home/dtch1997/github/mcx-lab/rl-baselines3-zoo')

# Inspect logged inputs / outputs

Logs generated with `scripts/enjoy_with_logging.py`

In [27]:
import numpy as np
import pickle 
import json

def load_robot_states(stats_dir):
    base_rpy = np.load(stats_dir / 'base_rpy.npy')
    base_position = np.load(stats_dir / 'base_position.npy')
    base_velocity = np.load(stats_dir / 'base_velocity.npy')
    motor_position = np.load(stats_dir / 'motor_position.npy')
    motor_velocity = np.load(stats_dir / 'motor_velocity.npy')
    ref_displacement = np.load(stats_dir / 'reference_displacement.npy')
    act_displacement = np.load(stats_dir / 'actual_displacement.npy')
    ref_foot_contacts = np.load(stats_dir / 'reference_foot_contact.npy')
    act_foot_contacts = np.load(stats_dir / 'actual_foot_contact.npy')
    nn_observations = np.load(stats_dir / 'nn_observations.npy')

    reward_component_values = np.load(stats_dir / 'reward_components.npy')
    with open(stats_dir / 'reward_component_names.json', 'r') as file:
        reward_component_names = json.load(file)
    print(nn_observations.shape)
    nn_actions = np.load(stats_dir / 'nn_actions.npy')
    time = np.load(stats_dir / 'time.npy')
    
    return {
        'time': time, 
        'base_rpy': base_rpy,
        'base_pos': base_position,
        'base_vel': base_velocity,
        'motor_pos': motor_position, 
        'motor_vel': motor_velocity, 
        'ref_displacement': ref_displacement,
        'act_displacement': act_displacement,
        'ref_foot_contacts': ref_foot_contacts, 
        'act_foot_contacts': act_foot_contacts, 
        'nn_obs': nn_observations,
        'nn_act': nn_actions,
        'reward_component_values': reward_component_values,
        'reward_component_names': reward_component_names
    }

def load_normalizer(model_dir):
    with open(model_dir / 'A1GymEnv-v0' / 'vecnormalize.pkl', 'rb') as file:
        normalizer = pickle.load(file)
    return normalizer

In [28]:
import matplotlib.pyplot as plt

NUM_TIMESTEPS = 200

google_blue = (66 / 256, 133 / 256, 244 / 256, 1)
google_red = (219 / 256, 68 / 256, 55 / 256, 1)
google_yellow = (244 / 256, 180 / 256, 0, 1)
google_green = (15 / 256, 157 / 256, 88 / 256, 1)

foot_names = ['FR', 'FL', 'RR', 'RL']
foot_colors = [google_blue, google_red, google_yellow, google_green]
    
def draw_up_arrow(ax, x):
    ax.arrow(x -0.4, -0.35, 0, 0.2, length_includes_head=True, head_width=0.6, head_length=0.04, color='black')

def draw_down_arrow(ax, x):
    ax.arrow(x -0.4, 0.35, 0, -0.2, length_includes_head=True, head_width=0.6, head_length=0.04, color='black')

def draw_spanning_arrow(ax, x1, y1, x2, y2, name=""):
    xmid = (x1 + x2) / 2 
    ymid = (y1 + y2) / 2

    xdiff = xmid - x1 
    ydiff = ymid - y2

    ax.text(xmid, ymid + 0.1, name, horizontalalignment='center', verticalalignment='center', fontsize=12, color='red')
    ax.arrow(xmid, ymid, xdiff, ydiff, color='red', length_includes_head=True, head_width=0.04, head_length=0.5)
    ax.arrow(xmid, ymid, -xdiff, -ydiff, color='red', length_includes_head=True, head_width=0.04, head_length=0.5)

def get_interpolate_value(x, y, x_query):
    idx_last = None
    x_last = None
    for idx_curr, x_curr in enumerate(x):
        if x_query > x_curr:
            idx_last = idx_curr
            x_last = x_curr
        else:
            break
    if x_last is None:
        return np.NaN
    if idx_curr >= len(x):
        return np.NaN
    y_last = y[idx_last]
    y_curr = y[idx_curr]
    
    return (y_curr * (x_curr - x_query) + y_last * (x_query - x_last)) / (x_curr - x_last) 

n_timesteps = 5000

def plot_xy_coordinates_distribution(ax, statss):
    xs = []
    ys = []
    survivals = []
    for stats in statss:
        base_pos = stats['base_pos']
        nrows = base_pos.shape[0]
        base_pos = np.pad(base_pos, ((0, n_timesteps - nrows), (0, 0)), constant_values=np.NaN)
        xs.append(base_pos[:,0])
        ys.append(base_pos[:,1])
        survivals.append(1 if nrows == n_timesteps else 0)

    survival_rate = np.mean(survivals)

    # Arbitrarily define x-indices for plotting
    x_plot = np.linspace(0.001, 60, num = 6000)
    y_mean = []
    y_std = []
    # Obtain the interpolated y-values to calculate mean and stdev
    for i, x_query in enumerate(x_plot):
        y_querys = []
        for x, y in zip(xs, ys):
            y_querys.append(get_interpolate_value(x, y, x_query.item()))
        y_mean.append(np.nanmean(y_querys))
        y_std.append(np.nanstd(y_querys))

    y_mean = np.array(y_mean)
    y_std = np.array(y_std)

    ax.plot(x_plot, y_mean, color=google_green)
    ax.axhline(0.0, color = 'blue', linestyle='--')
    ax.fill_between(x_plot, y_mean + y_std, y_mean - y_std, color='tab:green', alpha=0.2)

    print("Survival rate: ", survival_rate)

# Generate plots for Heightfield environment

In [29]:
model_dir = project_dir / 'logs' / 'ppo' / 'A1GymEnv-v0_50'

statss = []
for seed in range(20):
    stats_dir = model_dir / f'throw-obj-stairs-seed{seed}'
    stats = load_robot_states(stats_dir)
    stats['name'] = f'seed{seed}'
    statss.append(stats)

(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(2349, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)
(5000, 1, 108)


In [32]:
import matplotlib.pyplot as plt

obstacle_pos = [7.5, 21.9, 36.3]
obstacle_names = ['3cm stairs', '5cm stairs', '7cm stairs']

def plot_arrows(ax):
    ax.set_xlim(0, 55)
    ax.set_ylim(-1.0, 1.0)    

    for i, start in enumerate(obstacle_pos):
        draw_spanning_arrow(ax, start, 0.7, start + 9, 0.7, name=obstacle_names[i]) 
        draw_up_arrow(ax, start)

fig, ax = plt.subplots(figsize=(10,2.5))
ax.set_xlabel('Forward distance travelled (m)')
ax.set_ylabel('Lateral deviation (m)')
# ax[1].set_ylabel('Survival rate')
# ax[1].set_ylim(0.0, 1.1)
plot_arrows(ax)
plot_xy_coordinates_distribution(ax, statss)
fig.tight_layout()
fig.savefig("robustness_thrown_stairs.pdf", format="pdf", bbox_layout="tight")